In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import sys
sys.path.append('../')

### Playground

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
s = set()
bool(s)

In [ ]:
df = pd.DataFrame({
    'x': np.random.uniform(-1.0, 1.0, size=(100,)).tolist(),
    'y': np.random.uniform(-1.0, 1.0, size=(100,)).tolist(),
    'flag': [random.choice([True, False]) for i in range(100)],
})
df.head()

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

In [ ]:
output_notebook()

In [ ]:
from bokeh.models import RangeSlider
from bokeh.layouts import column
from bokeh.events import SelectionGeometry
from hover.utils.bokeh_helper import servable

@servable()
def burner():
    """
    Trying to intersect the last manually specified selection with a slider coords/attribute range.
    """
    slider = RangeSlider(start=-1.0, end=1.0, value=(-0.5, 0.5), step=0.01)
    source = ColumnDataSource(df)
    plot = figure(tools=['poly_select', 'lasso_select', 'pan', 'wheel_zoom'])
    plot.circle(source=source)
    
    last_manual_selection = set()
    
    def subroutine(lower, upper):
        filter_l = set(np.where(df['y'] > lower)[0])
        filter_u = set(np.where(df['y'] < upper)[0])
        filtered = filter_l.intersection(filter_u)
        return filtered
    
    def selection_callback(event):
        """
        CAUTION: this has to overwrite the last manual selection.
        Hence only manual selections should trigger this callback.
        """
        last_manual_selection.clear()
        last_manual_selection.update(source.selected.indices.copy())
        filtered = subroutine(*slider.value)
        print('A')
        source.selected.indices = list(filtered.intersection(last_manual_selection))
        
    def foo(event):
        print('B')
        
    def slider_callback(attr, old, new):
        to_select = subroutine(*new)
        if last_manual_selection:
            to_select = to_select.intersection(last_manual_selection)
        source.selected.indices = list(to_select)
    
    plot.on_event(SelectionGeometry, selection_callback)
    plot.on_event(SelectionGeometry, foo)
    slider.on_change('value', slider_callback)
    
    return column(slider, plot)

In [ ]:
handle = burner()

In [ ]:
show(handle)

In [ ]:
from bokeh.io import show
from bokeh.models import CheckboxGroup, CustomJS

LABELS = ["Option 1", "Option 2", "Option 3"]

checkbox_group = CheckboxGroup(labels=LABELS, active=[0, 1])
checkbox_group.js_on_click(CustomJS(code="""
    console.log('checkbox_group: active=' + this.active, this.toString())
"""))

show(checkbox_group)

In [ ]:
from bokeh.models import Selection, RangeSlider, Button, Dropdown
from bokeh.layouts import row, column
from bokeh.io import output_notebook, show
from hover.utils.bokeh_helper import servable

output_notebook()

slider = RangeSlider(start=-1.0, end=1.0, value=(-0.5, 0.5), step=0.01)
slider.on_change("value", lambda attr, old, new: print(f"Range changed to {slider.value}"))

@servable()
def burner():
    arr = ['1', '2', '3']
    dropdown = Dropdown(
        label="Select Element",
        button_type="primary",
        menu=arr,
    )
    
    button = Button(label="Click Me", height=100)
    def button_callcack(event):
        dropdown.menu.append(str(int(dropdown.menu[-1]) + 1))
        print(f"Button Clicked! Expected menu: {arr}")
        print(f"Got menu: {dropdown.menu}")
    button.on_click(button_callcack)

    return column(dropdown, button)

handle = burner()
show(handle)

In [ ]:
from bokeh.events import ButtonClick, MenuItemClick

button_click = ButtonClick(button)

In [ ]:
button.subscribed_events

In [ ]:
button._trigger_event(button_click)

In [ ]:
help(slider._trigger_event)

In [ ]:
from bokeh.plotting import figure
from bokeh.events import SelectionGeometry
from hover.utils.bokeh_helper import servable

@servable()
def burner():
    def callback(event):
        if event.final:
            print('Hi')
            print(event.geometry)
        return None

    plot = figure(tools=["lasso_select", "poly_select"])
    plot.on_event(SelectionGeometry, callback)
    return plot

handle = burner()
show(handle)